In [19]:
import numpy as np
import plotly.graph_objects as go
import os
import pickle 
import pandas as pd

filepath = 'C:/Users/Michael Forsthofer/OneDrive - University of Sussex/BadenLab/2p-imaging/singleplane/St53/'
pkl_files = [f for f in os.listdir(filepath) if f.endswith('.pkl')]

roi_data = pd.DataFrame(columns=['stimulus', 'ot_depth', 'plane_depth', 'amplitude', 'size', 'xy_centroid','experiment_ID', 'stage'])



In [20]:
def pool_variable(input_data, stim):
    data_to_get = input_data[stim]
    if len(data_to_get)>0:
        flattened_data = np.hstack(data_to_get)
    else:
        flattened_data = np.array([])
    return flattened_data

#def measure_rois(inptud_data, stim):
    

In [21]:
ot_depth = []
amplitude = []
integral = []
latency = []
size = []
stimulus = []
plane_depth = []
experiment_ID = []
stage = []
x_centroid = []
y_centroid = []

for pkl_file in pkl_files:
    with open(filepath+pkl_file, 'rb') as f:
        extracted_data_loaded = pickle.load(f)
        
        stims = list(extracted_data_loaded['depths'].keys())
        cur_plane_depth = extracted_data_loaded['metadata']['depth']
        cur_stage = filepath.split('St')[-1].split('/')[0]
        
        for cur_stim in stims:

            cur_ot_depth = pool_variable(extracted_data_loaded['depths'], cur_stim)
            ot_depth.extend(cur_ot_depth)
            amplitude.extend(pool_variable(extracted_data_loaded['amplitudes'], cur_stim))
            size.extend(pool_variable(extracted_data_loaded['sizes'], cur_stim))
            x_centroid.extend(pool_variable(extracted_data_loaded['x_centroid'], cur_stim))
            y_centroid.extend(pool_variable(extracted_data_loaded['y_centroid'], cur_stim))
            integral.extend(pool_variable(extracted_data_loaded['activity_area'], cur_stim))
            latency.extend(pool_variable(extracted_data_loaded['latency'], cur_stim))
            stimulus.extend(np.repeat(np.array(cur_stim), len(cur_ot_depth)))
            plane_depth.extend(np.repeat(np.array(cur_plane_depth), len(cur_ot_depth)))
            experiment_ID.extend(np.repeat(np.array(pkl_file[11:26]), len(cur_ot_depth)))
            stage.extend(np.repeat(np.array(cur_stage), len(cur_ot_depth)))

In [22]:
pooled_data = pd.DataFrame({'ot_depth': ot_depth, 
                     'amplitude': amplitude, 
                     'size': size,
                     'x_centroid': x_centroid,
                     'integral': integral,
                     'latency': latency,
                     'y_centroid': y_centroid,
                     'stimtype': stimulus,
                     'plane_depth': plane_depth,
                     'experiment_ID': experiment_ID,
                     'stage': stage})

In [48]:
import plotly.graph_objects as go

fig = go.Figure()

# Loop through each unique value of 'stimtype'
for stim in pooled_data['stimtype'].unique():
    filtered_data = pooled_data[pooled_data['stimtype'] == stim]
    fig.add_trace(go.Histogram(y=filtered_data['ot_depth'], name=stim,
                               nbinsy=30
        ))  # Set nbinsx to desired number of bins

# Update the layout
fig.update_layout(barmode='overlay', yaxis_title='y_centroid', xaxis_title='Count', width=400, height=400, margin=dict(l=20, r=20, t=20, b=20),)
fig.update_traces(opacity=0.75)
#fig.update_xaxes(range=[0, 100])  # Disable gridlines for x-axis
#fig.update_yaxes(range=[-5,100])  # Disable gridlines for y-axis

fig.show()

In [52]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get the number of unique 'stimtype' values
num_stimtypes = pooled_data['stimtype'].nunique()

# Create a subplot with 1 row and 'num_stimtypes' columns
fig = make_subplots(rows=1, cols=num_stimtypes, shared_yaxes=True)

# Loop through each unique value of 'stimtype'
for i, stim in enumerate(pooled_data['stimtype'].unique(), start=1):
    filtered_data = pooled_data[pooled_data['stimtype'] == stim]
    fig.add_trace(go.Histogram(y=filtered_data['ot_depth'], name=stim,
                               nbinsy=30, opacity=0.75),
                  row=1, col=i)

# Update the layout
fig.update_layout(barmode='overlay', yaxis_title='y_centroid', xaxis_title='Count',
                  width=150 * num_stimtypes, height=300, margin=dict(l=20, r=20, t=20, b=20))

# Display the figure
fig.show()


In [24]:
import plotly.graph_objects as go

fig = go.Figure()

# Loop through each unique value of 'stimtype'
for stim in pooled_data['stimtype'].unique():
    filtered_data = pooled_data[pooled_data['stimtype'] == stim]
    fig.add_trace(go.Histogram(x=filtered_data['x_centroid'], name=stim, 
                               ybins = dict(  
            start=0,
            end=512,
            size=50)
        ))  # Set nbinsx to desired number of bins

# Update the layout
fig.update_layout(barmode='overlay', yaxis_title='x_centroid', xaxis_title='Count', width=600, height=300, margin=dict(l=20, r=20, t=20, b=20),)
fig.update_traces(opacity=0.75)
fig.update_xaxes(range=[0, 512])  # Disable gridlines for x-axis
#fig.update_yaxes(range=[0,100])  # Disable gridlines for y-axis

fig.show()

fig2 = go.Figure()

# Loop through each unique value of 'stimtype'
for stim in pooled_data['stimtype'].unique():
    filtered_data = pooled_data[pooled_data['stimtype'] == stim]
    fig2.add_trace(go.Histogram(y=filtered_data['y_centroid'], name=stim, 
                               ybins = dict(  
            start=0,
            end=128,
            size=10)
        ))  # Set nbinsx to desired number of bins

# Update the layout
fig2.update_layout(barmode='overlay', yaxis_title='y_centroid', xaxis_title='Count', width=400, height=400, margin=dict(l=20, r=20, t=20, b=20),)
fig2.update_traces(opacity=0.75)
#fig.update_xaxes(range=[0, 100])  # Disable gridlines for x-axis
fig.update_yaxes(range=[0,128])  # Disable gridlines for y-axis

fig2.show()

In [25]:
import plotly.express as px
fig = px.scatter(x=pooled_data['x_centroid'], y=pooled_data['y_centroid'], color=pooled_data['stimtype'], size=abs(pooled_data['amplitude']), opacity=pooled_data['size']/ pooled_data['size'].max())

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background for the plot area
    #paper_bgcolor='rgba(0,0,0,0)',  # Transparent background for the overall figure
)

# Update axes to remove gridlines
fig.update_xaxes(showgrid=False)  # Disable gridlines for x-axis
fig.update_yaxes(showgrid=False)  # Disable gridlines for y-axis

In [26]:
import plotly.express as px
import numpy as np
import pandas as pd

# Create empty lists to accumulate the data
x_vals = []
y_vals = []

# Loop to accumulate data
for id in np.unique(experiment_ID):
    for idx, stim in enumerate(np.unique(stimulus)):
        filtered_data = pooled_data[(pooled_data['stimtype'] == stim) & (pooled_data['experiment_ID'] == id)]
        responsive_area = filtered_data['size'].sum()
        x_vals.append(stim)
        y_vals.append(responsive_area)

# Create a DataFrame for easier calculation of means
data = pd.DataFrame({'x': x_vals, 'y': y_vals})

# Calculate the mean for each unique x value
means = data.groupby('x')['y'].mean().reset_index()

# Create the scatter plot after the loop
fig = px.scatter(x=x_vals, y=y_vals, labels={'y': 'Responsive Area [px]'})

# Update the layout and traces for the main scatter plot
fig.update_layout(width=300, height=400, margin=dict(l=20, r=20, t=20, b=20))
fig.update_traces(marker=dict(size=9, color='grey'))
fig.update_xaxes(range=[-1, 4])
fig.update_yaxes(range=[-100, 4000])

# Style the plot
fig.update_layout(
    plot_bgcolor='white',
    xaxis=dict(
        showline=True, linewidth=2, linecolor='black', mirror=False,
        tickvals=[0, 1, 2, 3],
        ticks='outside', ticklen=10, tickwidth=2, tickcolor='black'
    ),
    yaxis=dict(
        showline=True, linewidth=2, linecolor='black', mirror=False
    )
)

# Overlay black dots for the mean values
fig.add_scatter(
    x=means['x'], y=means['y'], 
    mode='markers', 
    marker=dict(size=12, color='crimson', symbol='circle'), 
    name='Mean'
)

# Show the plot
fig.show()


In [44]:
px.scatter(pooled_data, x='ot_depth', y='integral', color=pooled_data['stimtype'], opacity=0.9)

In [36]:
px.histogram(pooled_data, x='latency', color=pooled_data['stimtype'], opacity=0.7, nbins=40)